In [ ]:
# Mount "My Drive" into /content/drive
from google.colab import drive

private_dir = "Courses-VinBigdata/Course-Deep-Learning/Project/"
google_drive_dir = private_dir+"Final_Project_DL"  # @param
#bml-notebooks/
drive.mount('/content/drive')

mount_point = "/content/drive/MyDrive/{}".format(google_drive_dir)

# Change the root directory to your mount_point
% cd '$mount_point'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1BOvFtzoVhfYNe70lBGxYzhPlDElSrazo/Final_Project_DL


# <span style="color: royalblue;">Load Data</span>
Load the image data with labels.

In [ ]:
import numpy as np
from src.loadData import loadFer2013
from keras.utils import to_categorical


In [ ]:
X_train,X_val,X_test,y_train,y_val,y_test = loadFer2013('fer2013/icml_face_data.csv')

y_train_onehot = to_categorical(y_train)
y_val_onehot = to_categorical(y_val)
y_test_onehot = to_categorical(y_test)

In [ ]:
X_train.shape, y_train.shape

((28709, 48, 48), (28709,))

In [ ]:
emotions = {0: 'Angry', 1:'Disgust', 2:'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
class_names = list(emotions.values())
class_names

['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

#Imbalance Data

In [ ]:
import imblearn

if imblearn.__version__.split('.')[1] != '7':
   !pip install -U imbalanced-learn

In [ ]:
def SMOTE_data(X_train,y_train,mode='smote'):
    if mode=='random':
        oversample = imblearn.over_sampling.RandomOverSampler(sampling_strategy='auto')
    else:
        oversample = imblearn.over_sampling.SMOTE()

    X_train = X_train.reshape(X_train.shape[0], -1)

    X_train, y_train = oversample.fit_resample(X_train, y_train)

    X_train = X_train.reshape((X_train.shape[0], 48, 48))

    return X_train, y_train

In [ ]:
X_train_smote,y_train_smote= SMOTE_data(X_train,y_train,mode='smote')

In [ ]:
y_train_onehot = to_categorical(y_train_smote)

In [ ]:
y_train_onehot.shape

(50505, 7)

#Preprocessing

In [ ]:
def preprocessX(X):
    X = X.astype(np.float32)
    X = X/255
    X = np.repeat(X[..., np.newaxis], 3, -1)
    return X

X_processed_train = preprocessX(X_train_smote)
X_processed_val = preprocessX(X_val)
X_processed_test = preprocessX(X_test)
X_processed_train.shape,X_processed_val.shape,X_processed_test.shape

((50505, 48, 48, 3), (3589, 48, 48, 3), (3589, 48, 48, 3))

# RESNET 50

In [ ]:
# ResNet50
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.optimizers import Adam

model1 = ResNet50(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

x = model1.output
x= Flatten()(x)
x = Dense(7, activation='softmax')(x)
model50 = Model(inputs=model1.input, outputs=x)


model50.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model50.fit(X_processed_train, y_train_onehot, batch_size=64, epochs=40, steps_per_epoch=len(X_train)/128, validation_data=(X_processed_val, y_val_onehot), verbose=1)
model50.save('CNNmodel/SMOTE_RESNET50.h5')


Epoch 1/40
224/224 [==============================] - 29s 78ms/step - loss: 2.5153 - accuracy: 0.3285 - val_loss: 2.0868 - val_accuracy: 0.1432
Epoch 2/40
224/224 [==============================] - 16s 72ms/step - loss: 1.3487 - accuracy: 0.5075 - val_loss: 2.3375 - val_accuracy: 0.1641
Epoch 3/40
224/224 [==============================] - 16s 70ms/step - loss: 1.2057 - accuracy: 0.5516 - val_loss: 1.9942 - val_accuracy: 0.2251
Epoch 4/40
224/224 [==============================] - 16s 72ms/step - loss: 1.1080 - accuracy: 0.5883 - val_loss: 1.7620 - val_accuracy: 0.3402
Epoch 5/40
224/224 [==============================] - 16s 70ms/step - loss: 1.1746 - accuracy: 0.5808 - val_loss: 1.8959 - val_accuracy: 0.4589
Epoch 6/40
224/224 [==============================] - 16s 71ms/step - loss: 1.0268 - accuracy: 0.6160 - val_loss: 1.4425 - val_accuracy: 0.4703
Epoch 7/40
224/224 [==============================] - 16s 72ms/step - loss: 1.0126 - accuracy: 0.6325 - val_loss: 1.5347 - val_accuracy:

In [ ]:
model50.evaluate(X_processed_test,y_test_onehot)

113/113 [==============================] - 2s 13ms/step - loss: 1.6574 - accuracy: 0.5848


[1.6573500633239746, 0.5848425626754761]